In [1]:
!pip install ensemble_boxes

  Obtaining dependency information for numba from https://files.pythonhosted.org/packages/ed/13/b66627125b35f2987bd9872cf028b5e1e1ffcbc8d1e182ac4e84eed3998f/numba-0.58.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
  Obtaining dependency information for llvmlite<0.42,>=0.41.0dev0 from https://files.pythonhosted.org/packages/57/7d/ef28d5812f852b93bd2a583d00cdcde56833d31b645ae0eaa7e71eecfb4e/llvmlite-0.41.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 67.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 31.0 MB/s eta 0:00:00:00:0100:01


In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [13]:
# ensemble csv files
submission_files = ['/data/ephemeral/home/sample_submission/dino234+cascade_ensemble_wbf.csv',
                    '/data/ephemeral/home/sample_submission/YOLOv8n_ensemble_wbf_iou9.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [14]:
image_ids = submission_df[0]['image_id'].tolist()
print(len(submission_df))

2


In [15]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '../../dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [16]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
#weights = [1.0,0.7,0.3,0.3,1.2,1.2]

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()

        if len(predict_list)==0 or len(predict_list)==1:
            continue

        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []

        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0])
            box[1] = float(box[1])
            box[2] = float(box[2])
            box[3] = float(box[3])
            box_list.append(box)

        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        #boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,  iou_thr=iou_thr)
        for boxes, scores, labels in zip(boxes_list, scores_list, labels_list):
            for box, score, label in zip(boxes, scores, labels):
                label = int(label) 
                prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '

    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [17]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('../../sample_submission/dino234+cascade+YOLO_ensemble_wbf_++.csv', index=None)

submission.head()

,PredictionString,image_id
0,7 0.9635725021362305 213.8626251220703 50.0466...,test/0000.jpg
1,5 0.8724842071533203 133.7473602294922 0.01667...,test/0001.jpg
2,1 0.6486684679985046 423.69500732421875 268.30...,test/0002.jpg
3,9 0.6772239804267883 143.8419647216797 246.821...,test/0003.jpg
4,0 0.5448459386825562 424.0182800292969 409.903...,test/0004.jpg
